In [1]:
import numpy as np
import cv2
import os

In [2]:
# ==========================================
# 설정
# ==========================================
CALIB_FILE = "stereo_calib.npz"
CAMERA_INDEX = 0
FRAME_WIDTH = 1280
FRAME_HEIGHT = 480
# ==========================================

# 1. 캘리브레이션 데이터 로드
if not os.path.exists(CALIB_FILE):
    print(f"에러: '{CALIB_FILE}' 파일이 없습니다.")
    exit()

data = np.load(CALIB_FILE)
mtxL, distL = data['mtxL'], data['distL']
mtxR, distR = data['mtxR'], data['distR']
R, T = data['R'], data['T']

In [3]:
# 2. 카메라 설정
cap = cv2.VideoCapture(CAMERA_INDEX)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
half_width = width // 2

# 3. Rectification 계산
# alpha=0: 검은 영역 제거 (Zoom in), alpha=1: 전체 유지
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtxL, distL, mtxR, distR, (half_width, height), R, T, alpha=0
)

map1_L, map2_L = cv2.initUndistortRectifyMap(mtxL, distL, R1, P1, (half_width, height), cv2.CV_16SC2)
map1_R, map2_R = cv2.initUndistortRectifyMap(mtxR, distR, R2, P2, (half_width, height), cv2.CV_16SC2)

# ★ 중요: 거리 계산을 위한 파라미터 추출
# P1[0,0]은 수평 초점거리(focal length)입니다.
focal_length = P1[0, 0] 
# T[0]은 베이스라인(두 카메라 간 거리)입니다. 단위는 캘리브레이션 때 쓴 단위(mm)입니다.
baseline = abs(T[0]) 

print(f"초점거리(f): {focal_length:.2f}, 베이스라인(B): {baseline.item():.2f}")

# 4. StereoSGBM (깊이 계산 엔진) 초기화
# 트랙바 조절을 위해 초기값 설정
min_disp = 0
num_disp = 16 * 6  # 16의 배수여야 함
block_size = 5     # 홀수여야 함 (3~11)

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size**2,
    P2=32 * 3 * block_size**2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)

# 5. UI 및 마우스 콜백 함수
cv2.namedWindow('Depth Map')

# 마우스 클릭 좌표 저장용
mouse_x, mouse_y = -1, -1

def on_mouse(event, x, y, flags, param):
    global mouse_x, mouse_y
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_x, mouse_y = x, y

cv2.setMouseCallback('Depth Map', on_mouse)

# 트랙바 콜백 (아무것도 안 함, 값 읽기용)
def nothing(x):
    pass

# 튜닝용 트랙바 생성
cv2.createTrackbar('Num Disparity', 'Depth Map', 6, 20, nothing) # 값 * 16이 실제 numDisp
cv2.createTrackbar('Block Size', 'Depth Map', 5, 50, nothing)    # 실제 BlockSize

print("\n[사용법]")
print("- 'Depth Map' 창의 트랙바를 조절해 화질을 개선하세요.")
print("- 화면의 물체를 마우스로 클릭하면 거리가 표시됩니다.")
print("- 'q' 키: 종료")

while True:
    ret, frame = cap.read()
    if not ret: break

    # 1. Rectification (보정)
    imgL_raw = frame[:, :half_width]
    imgR_raw = frame[:, half_width:]
    
    imgL = cv2.remap(imgL_raw, map1_L, map2_L, cv2.INTER_LINEAR)
    imgR = cv2.remap(imgR_raw, map1_R, map2_R, cv2.INTER_LINEAR)

    # 2. 트랙바 값 읽어서 파라미터 업데이트
    # Num Disparities는 16의 배수여야 하므로 16을 곱해줍니다.
    n_disp_val = cv2.getTrackbarPos('Num Disparity', 'Depth Map') * 16
    blk_size_val = cv2.getTrackbarPos('Block Size', 'Depth Map')
    
    if n_disp_val < 16: n_disp_val = 16
    if blk_size_val % 2 == 0: blk_size_val += 1 # 홀수만 가능
    if blk_size_val < 3: blk_size_val = 3

    stereo.setNumDisparities(n_disp_val)
    stereo.setBlockSize(blk_size_val)

    # 3. Disparity 계산
    # 결과는 16배 스케일된 값(fixed-point)이 나옵니다.
    disp_raw = stereo.compute(imgL, imgR).astype(np.float32)

    # 4. 시각화를 위한 정규화 (보기 좋게 만듦)
    # disp_raw를 바로 쓰면 어둡게 보임 -> 0~255로 펴줌
    disp_vis = cv2.normalize(disp_raw, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    disp_color = cv2.applyColorMap(disp_vis, cv2.COLORMAP_JET)

    # 5. 마우스 클릭 지점 거리 계산
    if mouse_x > 0 and mouse_y > 0:
        # 16으로 나누어 실제 disparity 픽셀 값을 구함
        d_val = disp_raw[mouse_y, mouse_x] / 16.0
        
        if d_val > 0:
            # 거리 공식: Z = (f * B) / d
            distance = (focal_length * baseline) / d_val
            dist_str = f"{distance:.1f} mm"
        else:
            dist_str = "Unknown" # 시차가 0이거나 노이즈인 경우

        # 화면에 표시
        cv2.circle(disp_color, (mouse_x, mouse_y), 5, (255, 255, 255), 2)
        cv2.putText(disp_color, dist_str, (mouse_x + 10, mouse_y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # 6. 결과 출력
    # 왼쪽은 실제 영상, 오른쪽은 깊이 맵
    cv2.imshow("Original (Left Rectified)", imgL)
    cv2.imshow("Depth Map", disp_color)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

초점거리(f): 471.32, 베이스라인(B): 59.70

[사용법]
- 'Depth Map' 창의 트랙바를 조절해 화질을 개선하세요.
- 화면의 물체를 마우스로 클릭하면 거리가 표시됩니다.
- 'q' 키: 종료
